In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

In [29]:
# parameters
customer_id = '90'
formatted_attribute = 'Binder Style'
customer_name='%wbmason%'
buckets = """Office Chairs"""

In [30]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [31]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    
#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

In [34]:
import json
df = query_from_file(file_name='./query/custom_fields.sql', params=params)
# df['custom_fields'] = df['custom_fields'].apply(lambda x: json.loads(x))
custom_field_df=pd.json_normalize(df['custom_fields'])
# # custom_field_df = pd.json_normalize(df['custom_fields'])
fields = ['Binder Style']
length = pd.concat([df.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(length))

2268


In [35]:
# [*custom_field_df]

In [38]:
print(len(length[(length['Binder Style'].astype(str)!='nan')&(length['Binder Style'].astype(str)!='[]')&(length['Binder Style'].astype(str)!='NaN')]))
length[(length['Binder Style'].astype(str)!='nan')&(length['Binder Style'].astype(str)!='[]')&(length['Binder Style'].astype(str)!='NaN')]['Binder Style'].explode().value_counts()#[length['Theme'].astype(str)=='Coloring']  #(length['Theme'].astype(str)!='[]')|(length['Theme'].astype(str)!='nan')|(length['Theme'].astype(str)!='NaN')]                

603


View         365
Non-View     231
Aluminum       5
Specialty      2
Name: Binder Style, dtype: int64

In [73]:
regex_pattern_length=r'''(?i)(.{0,40}paper.{0,10})|()'''         
length['match'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

regex_pattern_length=r'''(?i)(construction.?paper)|()'''         
length['construction'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

regex_pattern_length=r'''(?i)(color.{0,15}paper)|()'''         
length['color'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

regex_pattern_length=r'''(?i)(copy.{0,5}paper)|()'''         
length['copy'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

regex_pattern_length=r'''(?i)(matte.{0,5}paper)|()'''         
length['matte'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

regex_pattern_length=r'''(?i)(printout.{0,5}paper)|()'''         
length['printout'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

regex_pattern_length=r'''(?i)(Cardstock)|()'''         
length['cardstock'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

regex_pattern_length=r'''(?i)(digital.?paper)|()'''         
length['digital'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

regex_pattern_length=r'''(?i)(bond.?paper)|()'''         
length['bond'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

regex_pattern_length=r'''(?i)(format.?paper)|()'''         
length['wide_large'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

print(len(df))
print('blanks: '+str(len(length[(length['digital'].astype(str)=='[]')&(length['bond'].astype(str)=='[]')&(length['wide_large'].astype(str)=='[]')&(length['matte'].astype(str)=='[]')&(length['copy'].astype(str)=='[]')&(length['color'].astype(str)=='[]')&(length['construction'].astype(str)=='[]')&(length['cardstock'].astype(str)=='[]')&(length['printout'].astype(str)=='[]')])))
print('matches: '+str(len(length[(length['digital'].astype(str)!='[]')|(length['bond'].astype(str)!='[]')|(length['wide_large'].astype(str)!='[]')|(length['matte'].astype(str)!='[]')|(length['copy'].astype(str)!='[]')|(length['color'].astype(str)!='[]')|(length['construction'].astype(str)!='[]')|(length['cardstock'].astype(str)!='[]')|(length['printout'].astype(str)!='[]')])))                       
length[(length['digital'].astype(str)!='[]')|(length['bond'].astype(str)!='[]')|(length['wide_large'].astype(str)!='[]')|(length['matte'].astype(str)!='[]')|(length['copy'].astype(str)!='[]')|(length['color'].astype(str)!='[]')|(length['construction'].astype(str)!='[]')|(length['cardstock'].astype(str)!='[]')|(length['printout'].astype(str)!='[]')][0:500]


2784
blanks: 1463
matches: 1321


buckets  bucket_id  product_id external_id  \
0     Papers       8592   434379558  5637178195   
1     Papers       8592   434379569  5637178196   
2     Papers       8592   434379570  5637178197   
3     Papers       8592   434379575  5637178198   
4     Papers       8592   434379581  5637178199   
5     Papers       8592   434379586  5637178200   
6     Papers       8592   434379591  5637178203   
7     Papers       8592   434379596  5637178204   
8     Papers       8592   434379736  5637176596   
9     Papers       8592   434379738  5637176597   
10    Papers       8592   434379739  5637176598   
11    Papers       8592   434379740  5637176600   
12    Papers       8592   434379741  5637176601   
13    Papers       8592   434379744  5637176606   
14    Papers       8592   434379745  5637176607   
15    Papers       8592   434379746  5637176608   
16    Papers       8592   434379748  5637176609   
17    Papers       8592   434379749  5637176610   
18    Papers       8592   434379751  5637176612   
19    Papers       8592   434379753  5637176613   
20    Papers       8592   434379754  5637176614   
21    Papers       8592   434380093  5637179689   
22    Papers       8592   434380094  5637179690   
23    Papers       8592   434380095  5637179691   
24    Papers       8592   434380096  5637179692   
25    Papers       8592   434380098  5637179693   
26    Papers       8592   434380099  5637179694   
27    Papers       8592   434380100  5637179695   
28    Papers       8592   434380101  5637179696   
29    Papers       8592   434380130  5637180943   
30    Papers       8592   434380131  5637180944   
31    Papers       8592   434380132  5637180945   
32    Papers       8592   434380133  5637180946   
33    Papers       8592   434380134  5637180947   
34    Papers       8592   434380135  5637180948   
35    Papers       8592   434380136  5637180949   
36    Papers       8592   434380137  5637180951   
37    Papers       8592   434380138  5637180952   
38    Papers       8592   434380139  5637180953   
39    Papers       8592   434380140  5637180954   
40    Papers       8592   434380141  5637180955   
41    Papers       8592   434380142  5637180956   
42    Papers       8592   434380143  5637180957   
45    Papers       8592   434380411  5637176440   
47    Papers       8592   434380413  5637176441   
48    Papers       8592   434380414  5637176442   
49    Papers       8592   434380415  5637176443   
50    Papers       8592   434380416  5637176444   
51    Papers       8592   434380418  5637176451   
53    Papers       8592   434380583  5637178206   
54    Papers       8592   434380584  5637178207   
66    Papers       8592   434380710  5637179265   
68    Papers       8592   434380822  5637180886   
69    Papers       8592   434380823  5637180887   
73    Papers       8592   434381245  5637178178   
74    Papers       8592   434381246  5637178180   
75    Papers       8592   434381247  5637178181   
76    Papers       8592   434381252  5637178182   
77    Papers       8592   434381253  5637178183   
78    Papers       8592   434381261  5637178184   
79    Papers       8592   434381266  5637178185   
80    Papers       8592   434381267  5637178186   
81    Papers       8592   434381277  5637178189   
82    Papers       8592   434381284  5637178190   
83    Papers       8592   434381285  5637178191   
84    Papers       8592   434381286  5637178192   
85    Papers       8592   434381291  5637178193   
86    Papers       8592   434381292  5637178194   
87    Papers       8592   434381294  5637178164   
88    Papers       8592   434381295  5637178165   
89    Papers       8592   434381296  5637178166   
90    Papers       8592   434381297  5637178167   
91    Papers       8592   434381298  5637178168   
92    Papers       8592   434381299  5637178169   
93    Papers       8592   434381300  5637178172   
94    Papers       8592   434381301  5637178173   
95    Papers       8592   434381302  5637178175   
96    Papers       8592   4

In [43]:
regex_pattern_length=r'''(?i)(.{0,20}\d+(?:\.\d+|.\d+\/\d+).{0,3}x.?\d+\,?(?:\d+|\.\d+"?|.\d+\/\d+)\'.{0,20})|()'''         
length['match'] = length['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

trip=r'''(?i)((?<!\.)\d+\,\d+.)|((?<!\.)\d\d\d.)|()'''         
length['matches'] = length['match'].apply(lambda x: re_extract(trip, str(x)))

print(len(length))
print(len(length[length['match'].astype(str)!='[]']))
roll=length[length['match'].astype(str)=='[]']
roll[curation_col]='n/a'
matchna=roll[['external_id',curation_col]]
matchna

4059
40


external_id Q:roll_length
0     5637348496           n/a
1     5637350089           n/a
2     5637369393           n/a
3     5637369421           n/a
4     5637369422           n/a
...          ...           ...
4054  5637439860           n/a
4055  5637439863           n/a
4056  5637440930           n/a
4057  5637499477           n/a
4058  5637439864           n/a

[4019 rows x 2 columns]

In [12]:
inches=r'''(?i)(8\.5.{0,3}x\s?11)|(8.?1\/2.{0,3}x\s?11)|()'''
length['eigth']=length['long_desc'].apply(lambda x: re_extract(inches, str(x)))
print(len(length[length['eigth'].astype(str)!='[]']))
eight=length[length['eigth'].astype(str)!='[]']
eight[curation_col]='11 in'
matcheight=eight[['external_id',curation_col]]
matcheight

2470


external_id Q:length
68     5637177273    11 in
69     5637177274    11 in
70     5637177275    11 in
71     5637177276    11 in
111    5637177145    11 in
...           ...      ...
14593  5637505070    11 in
14594  5637505071    11 in
14595  5637505072    11 in
14596  5637505073    11 in
14597  5637505074    11 in

[2470 rows x 2 columns]

In [13]:
df['buckets'].explode().value_counts()[0:10]

Papers                                             3095
Folders & File Bags                                1970
Binders & Accessories                              1296
Labels, Stickers & Tickets                         1150
Notebooks, Journals, & Writing Pads                 613
Presentation Boards                                 608
Calendars                                           563
Food Storage Canisters/Containers & Accessories     386
Forms & Logbooks                                    374
Greeting Cards, Postcards, & Invitations            358
Name: buckets, dtype: int64

In [21]:
print(len(length[length['eigth'].astype(str)=='[]']))

inches=r'''(?i)(.{0,10}\d+.{0,3}x\s?\d+.{0,10})|(\d+["'](?!.Cap))|()'''
length['general']=length['long_desc'].apply(lambda x: re_extract(inches, str(x)))

print(len(length[(length['buckets'].astype(str)=='Binders & Accessories')&(length['general'].astype(str)=='[]')][0:500]))
match=length[(length['buckets'].astype(str)=='Binders & Accessories')&(length['general'].astype(str)=='[]')
match[curation_col]='n/a'
matchone=match[['external_id',curation_col]]
matchone

12512
117


external_id Q:length
30    5637178506      n/a
31    5637178507      n/a
32    5637178508      n/a
33    5637178515      n/a
34    5637178528      n/a
35    5637178529      n/a
36    5637178530      n/a
37    5637178531      n/a
38    5637178532      n/a
39    5637178533      n/a
40    5637178550      n/a
41    5637178574      n/a
42    5637178575      n/a
43    5637178577      n/a
44    5637178580      n/a
45    5637178587      n/a
46    5637178589      n/a
47    5637178594      n/a
126   5637178596      n/a
127   5637178600      n/a
128   5637178605      n/a
129   5637178612      n/a
130   5637178613      n/a
131   5637178617      n/a
135   5637178652      n/a
244   5637213547      n/a
330   5637214203      n/a
471   5637283800      n/a
472   5637284154      n/a
501   5637283977      n/a
502   5637284315      n/a
503   5637284381      n/a
506   5637283907      n/a
507   5637283922      n/a
509   5637284298      n/a
518   5637300514      n/a
602   5637329248      n/a
603   5637329254      n/a
604   5637338774      n/a
605   5637338775      n/a
606   5637338776      n/a
607   5637338777      n/a
611   5637338891      n/a
612   5637338892      n/a
613   5637338893      n/a
614   5637338894      n/a
615   5637338967      n/a
616   5637338968      n/a
617   5637338969      n/a
618   5637338970      n/a
619   5637338972      n/a
620   5637338973      n/a
621   5637338974      n/a
623   5637338778      n/a
624   5637338779      n/a
625   5637338782      n/a
626   5637338783      n/a
627   5637338784      n/a
628   5637338785      n/a
629   5637338786      n/a
630   5637338787      n/a
631   5637338788      n/a
632   5637338790      n/a
633   5637338886      n/a
634   5637338887      n/a
635   5637338888      n/a
636   5637338889      n/a
659   5637355400      n/a
748   5637387364      n/a
762   5637408037      n/a
877   5637145252      n/a
878   5637145251      n/a
879   5637145253      n/a
880   5637145258      n/a
881   5637145259      n/a
992   5637145178      n/a
993   5637145179      n/a
994   5637145180      n/a
995   5637145181      n/a
996   5637145182      n/a
997   5637145183      n/a
998   5637145184      n/a
999   5637145185      n/a
1000  5637145186      n/a
1001  5637145187      n/a
1002  5637145188      n/a
1003  5637145190      n/a
1004  5637145191      n/a
1005  5637145192      n/a
1006  5637145193      n/a
1007  5637145194      n/a
1008  5637145195      n/a
1009  5637145196      n/a
1010  5637145197      n/a
1011  5637145198      n/a
1012  5637145199      n/a
1013  5637145200      n/a
1014  5637145201      n/a
1015  5637145202      n/a
1016  5637145203      n/a
1018  5637145205      n/a
1019  5637145206      n/a
1020  5637145207      n/a
1022  5637145209      n/a
1023  5637145210      n/a
1024  5637145211      n/a
1025  5637145212      n/a
1026  5637145213      n/a
1027  5637145214      n/a
1028  5637145215      n/a
1029  5637145218      n/a
1034  5637145236      n/a
1047  5637145250      n/a
1124  5637150342      n/a
1207  5637487878      n/a
1208  5637487879      n/a
1209  5637487880      n/a

In [44]:
# # def get_df_name(df):
# #     name =[x for x in globals() if globals()[x] is df][0]
# #     return name
# # print(get_df_name(matchsplitinch))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchcustom))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchesna))


# # def get_df_name(df):
# #     name =[x for x in globals() if globals()[x] is df][0]
# #     return name
# # print(get_df_name(matchesheight))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(matcheight))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(matchna))

matchna


In [ ]:
stop

# send to the folder for upload

In [46]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchna)